- [참고 data2vec audio:사용 안하기로함]: https://towhee.io/audio-embedding/data2vec
- [참고 data2vec text]: https://huggingface.co/docs/datasets/nlp_load
- [참고 data2vec load audio*]:https://huggingface.co/docs/datasets/audio_load
- [참고 data2vec embedding]: https://huggingface.co/docs/transformers/model_doc/data2vec#transformers.Data2VecAudioModel
- [참고 handling pytorch dataset]:https://stanford.edu/~shervine/blog/pytorch-how-to-generate-data-parallel
- [참고 add data to dataset]: https://pypi.org/project/datasets/

# Embeddings(Text, Audio) Session 1 
- 각세션으로 확대시켜야 함

In [2]:
import torch
from transformers import AutoProcessor, Data2VecAudioModel
from datasets import load_dataset, Dataset, Audio, Features
from glob import glob


/home/arplab/project/lou/multi_modal/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Audio Embedding

In [3]:
# from towhee import pipe, ops, DataCollection # to be added
# p = (pipe.input('path')
#         .map('path', 'frame', ops.audio_decode.ffmpeg(sample_rate=16000))
#         .map('frame', 'vecs', ops.audio_embedding.data2vec(model_name='facebook/data2vec-audio-base-960h'))
#         .output('path', 'vecs'))

# DataCollection(p('./org_KEMDy20/Session01/Sess01_script01_User001F_001.wav')).show()

In [4]:
wav_file_lst = glob('./org_KEMDy20/Session01/*.wav')
wav_file_lst[:3]

['./org_KEMDy20/Session01/Sess01_script03_User002M_010.wav',
 './org_KEMDy20/Session01/Sess01_script03_User001F_014.wav',
 './org_KEMDy20/Session01/Sess01_script05_User002M_001.wav']

In [5]:
import datasets

dataset = Dataset.from_dict({"wav": wav_file_lst }).cast_column("wav", Audio())
dataset[0]["wav"]

{'path': './org_KEMDy20/Session01/Sess01_script03_User002M_010.wav',
 'array': array([-0.00018311,  0.00057983,  0.00076294, ...,  0.00259399,
         0.00115967,  0.00036621], dtype=float32),
 'sampling_rate': 16000}

In [6]:
check_point_wav_model = 'facebook/data2vec-audio-base-960h'
processor = AutoProcessor.from_pretrained(check_point_wav_model)
model = Data2VecAudioModel.from_pretrained(check_point_wav_model)

Some weights of the model checkpoint at facebook/data2vec-audio-base-960h were not used when initializing Data2VecAudioModel: ['lm_head.weight', 'lm_head.bias']
- This IS expected if you are initializing Data2VecAudioModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Data2VecAudioModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
dataset[0]['wav']['array']

array([-0.00018311,  0.00057983,  0.00076294, ...,  0.00259399,
        0.00115967,  0.00036621], dtype=float32)

In [8]:
# def make_batch(batch_size, dataset, column = None):
#     data_lst = []
#     if column == None:
#         column = 'wav'
#     for i in range(batch_size-1):
#         data_lst.append(dataset[i]['wav']['array'])
#     return data_lst

In [9]:
sampling_rate = dataset['wav'][0]['sampling_rate']

In [10]:
input_values = []
for i in range(len(dataset)): 
    inputs = processor(dataset[i]['wav']['array'], sampling_rate = sampling_rate, return_tensors = 'pt', padding = True, max_length=400)
    with torch.no_grad():
        output = model(**inputs)
    input_values.append(output['last_hidden_state'])

print(len(input_values))
input_values[:3] # wav embedded data


2023-03-27 14:46:10.038042: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-27 14:46:11.270795: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11/include:/usr/local/cuda-11/lib64::/usr/local/cuda/extras/CUPTI/lib64
2023-03-27 14:46:11.270915: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11/include:/usr/local/cuda

311


[tensor([[[-0.7484,  0.0591, -0.2074,  ...,  0.1284,  0.0253,  0.1848],
          [-0.7484,  0.0591, -0.2074,  ...,  0.1284,  0.0253,  0.1848],
          [-0.7484,  0.0591, -0.2074,  ...,  0.1284,  0.0253,  0.1848],
          ...,
          [-1.0762,  0.0578,  0.0159,  ..., -0.0998, -0.1490,  0.0300],
          [-1.0765,  0.0587,  0.0153,  ..., -0.0998, -0.1485,  0.0301],
          [-1.0873,  0.0756,  0.0079,  ..., -0.1036, -0.1474,  0.0286]]]),
 tensor([[[-0.9110,  0.1859, -0.2962,  ...,  0.1183,  0.0295,  0.3080],
          [-0.9110,  0.1859, -0.2962,  ...,  0.1183,  0.0295,  0.3080],
          [-0.9110,  0.1859, -0.2962,  ...,  0.1183,  0.0295,  0.3080],
          ...,
          [-0.9165,  0.1055, -0.0734,  ..., -0.1464, -0.1688,  0.1187],
          [-0.9164,  0.1068, -0.0735,  ..., -0.1464, -0.1669,  0.1192],
          [-0.9177,  0.1112, -0.0758,  ..., -0.1466, -0.1632,  0.1201]]]),
 tensor([[[-0.5680,  0.1276, -0.2977,  ...,  0.2255, -0.0380,  0.2583],
          [-0.5680,  0.1276,

In [11]:
# # audio file is decoded on the fly
# input_values = []
# for i in range(len(dataset)):
#     input = feature_extractor(dataset[i]["wav"]["array"], sampling_rate=dataset[0]['wav']['sampling_rate'], return_attention_mask=False)
#     input_values.append(input['input_values'][0])
    

input_values_feature = Features({'embedded_wav': input_values})
input_values_feature['embedded_wav'][:3]

[tensor([[[-0.7484,  0.0591, -0.2074,  ...,  0.1284,  0.0253,  0.1848],
          [-0.7484,  0.0591, -0.2074,  ...,  0.1284,  0.0253,  0.1848],
          [-0.7484,  0.0591, -0.2074,  ...,  0.1284,  0.0253,  0.1848],
          ...,
          [-1.0762,  0.0578,  0.0159,  ..., -0.0998, -0.1490,  0.0300],
          [-1.0765,  0.0587,  0.0153,  ..., -0.0998, -0.1485,  0.0301],
          [-1.0873,  0.0756,  0.0079,  ..., -0.1036, -0.1474,  0.0286]]]),
 tensor([[[-0.9110,  0.1859, -0.2962,  ...,  0.1183,  0.0295,  0.3080],
          [-0.9110,  0.1859, -0.2962,  ...,  0.1183,  0.0295,  0.3080],
          [-0.9110,  0.1859, -0.2962,  ...,  0.1183,  0.0295,  0.3080],
          ...,
          [-0.9165,  0.1055, -0.0734,  ..., -0.1464, -0.1688,  0.1187],
          [-0.9164,  0.1068, -0.0735,  ..., -0.1464, -0.1669,  0.1192],
          [-0.9177,  0.1112, -0.0758,  ..., -0.1466, -0.1632,  0.1201]]]),
 tensor([[[-0.5680,  0.1276, -0.2977,  ...,  0.2255, -0.0380,  0.2583],
          [-0.5680,  0.1276,

In [12]:
len(input_values_feature)

1

In [13]:
# dataset_new = dataset.add_column(name = "embedded_wav", column = input_values_feature['embedded_wav'])
# dataset_new

## Text Embedding


- 참고: https://huggingface.co/docs/datasets/nlp_load
- 참고: https://huggingface.co/docs/transformers/v4.27.2/en/model_doc/data2vec#transformers.Data2VecTextModel
- 참고 한국어 pretrained model: https://huggingface.co/Junmai/KR-Data2VecText-v1

In [14]:
from transformers import PreTrainedTokenizerFast, BartModel
from transformers import AutoModel, AutoTokenizer
import pandas as pd



In [15]:
# text data loading
txt_file_lst = glob('./org_KEMDy20/Session01/*.txt')
txt_file_lst[:3]

['./org_KEMDy20/Session01/Sess01_script06_User002M_009.txt',
 './org_KEMDy20/Session01/Sess01_script01_User001F_007.txt',
 './org_KEMDy20/Session01/Sess01_script01_User001F_001.txt']

In [16]:
# 참고: https://huggingface.co/gogamza/kobart-base-v2
# 참고: https://github.com/BM-K/Sentence-Embedding-Is-All-You-Need

# model = AutoModel.from_pretrained('gogamza/kobart-base-v2')  # or 'BM-K/KoSimCSE-bert-multitask'
# tokenizer = AutoTokenizer.from_pretrained('gogamza/kobart-base-v2')  # or 'BM-K/KoSimCSE-bert-multitask'
checkpoint = 'BM-K/KoSimCSE-bert-multitask'
model = AutoModel.from_pretrained(checkpoint)  # or 'BM-K/KoSimCSE-bert-multitask'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)  # or 'BM-K/KoSimCSE-bert-multitask'


In [19]:
import re
sentences = []

for i in txt_file_lst:    
    f = open(i, 'r')
    line = f.readline()
    line = re.sub('l/\n|c/|b/|n/|l/|u/', '', line)
    line = re.sub('  ', ' ', line)
    line = line.rstrip().lstrip()
    sentences.append(line)
    f.close()
    
    
sentences[21:40]

['그것 그것 땜에 당 그거 땜에 당황스러웠던 적은 없었어?',
 '아니면 어 어.',
 '그니까 그런 경험이 음 솔직히 거의 없는 거 같은데 약간 명절에 시골에 내려 가면은 약간 큰 아빠?',
 '그 가영이는 뭐 주변에서 막 이렇게 보고 가영이가 그거 했던 적은 웃겼던 건 적은 없었어?',
 '근데 내가 만약에 열두시에 연락을 해서 어 저 한시간 정도 더 놀다 갈게요 했는데 그 한시까지 안오잖아?',
 '이제 그거에 대해서는 어린 나이에도 대표를 다시는 여성분들도 계시는데 굳이 그거를 비율을 맞춰야 되느냐에 대해서 조금 그런 거 같애.',
 '근데 그거는 굳이 약간 남녀 그런 문제가 아니라 누가 도우려고 하든 간에 내가 도움을 받는 걸 싫어해서 그런 거야.',
 '어 좀 비슷하게 라고는 말하기 좀 그런데 약간 버스를 타다가 넘어 그니까 되게 급 정차를 해 하는 바람에 버스가 그래서 막.',
 '내가 혼잣말하거나 전화를 막 하고 있는데 집에서 자기 혼자 네 부르셨나요 막 이러거나 되게 띠링 소리 나다가 자기 혼자 갑자기 노래를 트는 경우가 있어.',
 '글쎄 일단은 나는 그 말이 살짝 좀 이해가 안 됐어, 약간.',
 '같은 아파트 내에 있는 어른들이나 그런 사람들한텐 다 인사하고 다녔어.',
 '말도 말도 안 했는데 그냥 막 떠.',
 '혹시 여기 무슨 검사 받았 적 있었냐고 나한테 검사받았던 환자시냐고 묻는 거야.',
 '그 꼰대에 특징을 갖고 있다거나 막 그런 거?',
 '그런 면에서 되게 아버지가 뭐라 하시거나?',
 '한 여섯시까지는 들어가야 된다라는 게 있었는데 그거 그 규칙이 불문율이라고 했어야 되나 근데 그거를 오빠가 깨버렸거든.',
 '어 최근에 어디를 가면?',
 '물론 같이 공부를 했을 때 똑같은 사람들도 있겠지만.',
 '우리 집은 되게 자유로워 가지고 솔직히 아빠가 이게 어때라고 하면 본인 생 그니까 가족들 생각이랑 안 맞으면 그냥 자기 주관대로 가거든, 다들.']

In [21]:
# option 1
inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")
txt_embeddings, _ = model(**inputs, return_dict=False)

# option 2
# 참고: https://discuss.huggingface.co/t/get-output-embedding-of-featureextractor/5636

# feature_extraction = pipeline('feature-extraction', model= model, tokenizer= tokenizer)
# features = feature_extraction("i am sentence")

In [22]:
len(txt_embeddings)

311